In [13]:
# !pip install datasets
# !pip install easyocr
# !pip install Pillow
# !pip install numpy
# !pip install matplotlib


ERROR: Could not find a version that satisfies the requirement torch==0.4.1.post2 (from versions: 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0)
ERROR: No matching distribution found for torch==0.4.1.post2


In [9]:
from datasets import load_dataset
import easyocr
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt

In [4]:
!git clone https://github.com/clovaai/CRAFT-pytorch.git

Cloning into 'CRAFT-pytorch'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 59 (delta 27), reused 24 (delta 24), pack-reused 23 (from 1)
Receiving objects: 100% (59/59), 1.69 MiB | 23.66 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [15]:
%cd /content/CRAFT-pytorch
!pip install -r requirements.txt

/content/CRAFT-pytorch


**Text Extraction**

> using easyocr

> support bahasa indonesia and english





In [16]:
from craft_text_detector import Craft

craft = Craft(output_dir='outputs/', crop_type="poly", cuda=False)  # Gunakan cuda=True jika pakai GPU

def detect_text_craft(image):
    # Jika gambar berupa PIL Image, konversi ke NumPy array
    if isinstance(image, Image.Image):
        image = np.array(image)

    # Jika gambar masih dalam mode grayscale atau alpha, ubah ke BGR
    if len(image.shape) == 2 or image.shape[2] == 1:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    # Jalankan deteksi teks
    prediction_result = craft.detect_text(image)

    # Ambil bounding boxes
    boxes = prediction_result["boxes"]

    # Gambar bounding box
    for box in boxes:
        pts = box.reshape((-1, 1, 2)).astype(int)
        cv2.polylines(image, [pts], isClosed=True, color=(0, 255, 0), thickness=2)

    # Tampilkan hasil
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()

    return boxes

ModuleNotFoundError: No module named 'craft_text_detector'

In [ ]:
reader = easyocr.Reader(['en', 'id'])

def is_receipt(text, logic='AND'):
    required_keywords = ['total', 'amount', 'jumlah', 'price', 'harga']
    text_lower = text.lower()

    #diubah nanti jadinya ada yang and ada yang or
    if logic == 'AND':
        return all(keyword in text_lower for keyword in required_keywords)
    elif logic == 'OR':
        return any(keyword in text_lower for keyword in required_keywords)
    else:
        raise ValueError("Invalid logic parameter. Use 'AND' or 'OR'.")


def process_image(image, logic='AND'):
    # Convert PIL Image to numpy array
    image_np = np.array(image)

    # Extract text using EasyOCR
    results = reader.readtext(image_np)

    # Combine all detected text into a single string
    extracted_text = ' '.join([result[1] for result in results])

    # Check if the extracted text is a receipt
    if is_receipt(extracted_text, logic=logic):
        print("This is a valid receipt.")
        print("Extracted Text:")
        print(extracted_text)
    else:
        print("This is not a valid receipt.")

In [ ]:
# Load the CORD-v2 dataset
dataset = load_dataset("naver-clova-ix/cord-v2")

# Example: Process the first few images in the dataset
for example in dataset['train'].select(range(5)):  # Process the first 5 examples
    image = example['image']  # Get the PIL Image object
    print("Processing image...")

    # Process the image with OR logic
    detect_text_east(image)
    process_image(image, logic='OR')
    print("-" * 50)

# Load international style dataset
dataset_international = load_dataset("katanaml-org/invoices-donut-data-v1")

# Example: Process the first few images in the dataset_international
for example_int in dataset_international['train'].select(range(5)):  # Process the first 5 examples
    image_int = example_int['image']
    print("Processing image international...")

    # Process the image with OR logic
    detect_text_east(image_int)
    process_image(image_int, logic='OR')
    print("-" * 50)